In [8]:
from seleniumbase import Driver

from rich.pretty import pprint
import json
import re

In [37]:
driver = Driver(uc=True, uc_cdp_events=True)

raw = []

def save(data):
    req = {}
    req["id"] = data["params"]["requestId"]
    req["path"] = data["params"]["headers"][":path"]
    if "/getData/" in req["path"]:
        raw.append(req)

driver.add_cdp_listener('Network.requestWillBeSentExtraInfo', save)

driver.get(
    "https://www.marinetraffic.com/en/ais/home/centerx:132.2/centery:43.0/zoom:10")

driver.sleep(3)

driver.click('button:contains("AGREE")')

driver.sleep(3)

for req in raw:
    try:
        response = driver.execute_cdp_cmd(
            "Network.getResponseBody", {"requestId": req["id"]})
        req["response"] = response
    except Exception as e:
        req["response"] = None

driver.sleep(1)
driver.close()

In [38]:
raw

[{'id': '97684.182',
  'path': '/getData/get_total_ships',
  'response': {'base64Encoded': False, 'body': '299127'}},
 {'id': '97684.193',
  'path': '/getData/get_data_json_4/z:10/X:444/Y:187/station:0',
  'response': {'base64Encoded': False,
   'body': '{"type":1,"data":{"rows":[{"LAT":"43.08003","LON":"132.319","SPEED":"0","COURSE":"332","HEADING":"226","ELAPSED":"3","DESTINATION":"","FLAG":"RU","LENGTH":"53","ROT":"0","SHIPNAME":"SEDANKA","SHIPTYPE":"2","SHIP_ID":"2818282","WIDTH":"10","L_FORE":"20","W_LEFT":"5","DWT":"329","GT_SHIPTYPE":"37"},{"LAT":"43.12085","LON":"132.3366","SPEED":"13","COURSE":"221","HEADING":"38","ELAPSED":"2","DESTINATION":"FOR ORDER","FLAG":"RU","LENGTH":"130","ROT":"0","SHIPNAME":"PLAVUCHIY KRAN 3500","SHIPTYPE":"3","SHIP_ID":"7203862","WIDTH":"48","L_FORE":"73","W_LEFT":"28","DWT":"15045","GT_SHIPTYPE":"127"},{"LAT":"43.22013","LON":"132.3126","SPEED":"18","COURSE":"51","HEADING":"277","ELAPSED":"335","DESTINATION":"CN ZOS>SUKHODOL","FLAG":"PA","LENGTH":"

In [41]:
def shipRawParser(raw):
    regex = r"z:(?P<z>\d+)\/X:(?P<x>\d+)\/Y:(?P<y>\d+)"

    shipData = []
    for r in raw:
        if "get_data_json_4" in r["path"]:
            matches = re.findall(regex, r["path"])
            z = matches[0][0]
            x = matches[0][0]
            y = matches[0][0]

            rows = json.loads(r["response"]["body"])["data"]["rows"]
            for row in rows:
                data = {}
                data["z"] = z
                data["x"] = x
                data["y"] = y

                data["data"] = row

                shipData.append(data)

    return shipData

shipData = shipRawParser(raw)

In [42]:
shipData

[{'z': '10',
  'x': '10',
  'y': '10',
  'data': {'LAT': '43.08003',
   'LON': '132.319',
   'SPEED': '0',
   'COURSE': '332',
   'HEADING': '226',
   'ELAPSED': '3',
   'DESTINATION': '',
   'FLAG': 'RU',
   'LENGTH': '53',
   'ROT': '0',
   'SHIPNAME': 'SEDANKA',
   'SHIPTYPE': '2',
   'SHIP_ID': '2818282',
   'WIDTH': '10',
   'L_FORE': '20',
   'W_LEFT': '5',
   'DWT': '329',
   'GT_SHIPTYPE': '37'}},
 {'z': '10',
  'x': '10',
  'y': '10',
  'data': {'LAT': '43.12085',
   'LON': '132.3366',
   'SPEED': '13',
   'COURSE': '221',
   'HEADING': '38',
   'ELAPSED': '2',
   'DESTINATION': 'FOR ORDER',
   'FLAG': 'RU',
   'LENGTH': '130',
   'ROT': '0',
   'SHIPNAME': 'PLAVUCHIY KRAN 3500',
   'SHIPTYPE': '3',
   'SHIP_ID': '7203862',
   'WIDTH': '48',
   'L_FORE': '73',
   'W_LEFT': '28',
   'DWT': '15045',
   'GT_SHIPTYPE': '127'}},
 {'z': '10',
  'x': '10',
  'y': '10',
  'data': {'LAT': '43.22013',
   'LON': '132.3126',
   'SPEED': '18',
   'COURSE': '51',
   'HEADING': '277',
   'EL

In [47]:
def shipDataParser(shipData):
  ships = {}

  for data in shipData:
    ship = {} | data["data"]
    ship["TILE_X"] = data["x"]
    ship["TILE_Y"] = data["y"]
    ship["TILE_Z"] = data["z"]

    ships[data["data"]["SHIP_ID"]] = ship

  return ships

ships = shipDataParser(shipData)

pprint(ships)

{
│   '2818282': {
│   │   'LAT': '43.08003',
│   │   'LON': '132.319',
│   │   'SPEED': '0',
│   │   'COURSE': '332',
│   │   'HEADING': '226',
│   │   'ELAPSED': '3',
│   │   'DESTINATION': '',
│   │   'FLAG': 'RU',
│   │   'LENGTH': '53',
│   │   'ROT': '0',
│   │   'SHIPNAME': 'SEDANKA',
│   │   'SHIPTYPE': '2',
│   │   'SHIP_ID': '2818282',
│   │   'WIDTH': '10',
│   │   'L_FORE': '20',
│   │   'W_LEFT': '5',
│   │   'DWT': '329',
│   │   'GT_SHIPTYPE': '37',
│   │   'TILE_X': '10',
│   │   'TILE_Y': '10',
│   │   'TILE_Z': '10'
│   },
│   '7203862': {
│   │   'LAT': '43.12085',
│   │   'LON': '132.3366',
│   │   'SPEED': '13',
│   │   'COURSE': '221',
│   │   'HEADING': '38',
│   │   'ELAPSED': '2',
│   │   'DESTINATION': 'FOR ORDER',
│   │   'FLAG': 'RU',
│   │   'LENGTH': '130',
│   │   'ROT': '0',
│   │   'SHIPNAME': 'PLAVUCHIY KRAN 3500',
│   │   'SHIPTYPE': '3',
│   │   'SHIP_ID': '7203862',
│   │   'WIDTH': '48',
│   │   'L_FORE': '73',
│   │   'W_LEFT': '28',
│   │   'DWT': '15045',
│   │   'GT_SHIPTYPE': '127',
│   │   'TILE_X': '10',
│   │   'TILE_Y': '10',
│   │   'TILE_Z': '10'
│   },
│   '463849': {
│   │   'LAT': '43.22013',
│   │   'LON': '132.3126',
│   │   'SPEED': '18',
│   │   'COURSE': '51',
│   │   'HEADING': '277',
│   │   'ELAPSED': '335',
│   │   'DESTINATION': 'CN ZOS>SUKHODOL',
│   │   'FLAG': 'PA',
│   │   'LENGTH': '182',
│   │   'ROT': '-33',
│   │   'SHIPNAME': 'CS FORTUNE',
│   │   'SHIPTYPE': '7',
│   │   'SHIP_ID': '463849',
│   │   'WIDTH': '28',
│   │   'L_FORE': '167',
│   │   'W_LEFT': '13',
│   │   'DWT': '34006',
│   │   'GT_SHIPTYPE': '9',
│   │   'TILE_X': '10',
│   │   'TILE_Y': '10',
│   │   'TILE_Z': '10'
│   },
│   '8649383': {
│   │   'LAT': '43.11924',
│   │   'LON': '132.3346',
│   │   'SPEED': '0',
│   │   'COURSE': '0',
│   │   'HEADING': None,
│   │   'ELAPSED': '482',
│   │   'DESTINATION': 'TEST',
│   │   'FLAG': 'RU',
│   │   'LENGTH': '257',
│   │   'SHIPNAME': 'VALENTIN PIKUL',
│   │   'SHIPTYPE': '8',
│   │   'SHIP_ID': '8649383',
│   │   'WIDTH': '34',
│   │   'L_FORE': '210',
│   │   'W_LEFT': '21',
│   │   'TILE_X': '10',
│   │   'TILE_Y': '10',
│   │   'TILE_Z': '10'
│   },
│   '7158248': {
│   │   'LAT': '42.89618',
│   │   'LON': '132.3602',
│   │   'SPEED': '0',
│   │   'COURSE': None,
│   │   'HEADING': '291',
│   │   'ELAPSED': '25',
│   │   'DESTINATION': 'RU VVO',
│   │   'FLAG': 'RU',
│   │   'LENGTH': '131',
│   │   'ROT': '0',
│   │   'SHIPNAME': 'ALEKSANDR DEEV',
│   │   'SHIPTYPE': '6',
│   │   'SHIP_ID': '7158248',
│   │   'WIDTH': '20',
│   │   'L_FORE': '81',
│   │   'W_LEFT': '5',
│   │   'DWT': '4829',
│   │   'GT_SHIPTYPE': '116',
│   │   'TILE_X': '10',
│   │   'TILE_Y': '10',
│   │   'TILE_Z': '10'
│   },
│   '753283': {
│   │   'LAT': '42.78976',
│   │   'LON': '132.7333',
│   │   'SPEED': '1',
│   │   'COURSE': '335',
│   │   'HEADING': '98',
│   │   'ELAPSED': '7',
│   │   'DESTINATION': 'FOR ORDERS',
│   │   'FLAG': 'RU',
│   │   'LENGTH': '249',
│   │   'ROT': '0',
│   │   'SHIPNAME': 'KRYMSK',
│   │   'SHIPTYPE': '8',
│   │   'SHIP_ID': '753283',
│   │   'WIDTH': '44',
│   │   'L_FORE': '210',
│   │   'W_LEFT': '22',
│   │   'DWT': '115663',
│   │   'GT_SHIPTYPE': '17',
│   │   'TILE_X': '10',
│   │   'TILE_Y': '10',
│   │   'TILE_Z': '10'
│   },
│   '8856752': {
│   │   'LAT': '42.81045',
│   │   'LON': '132.889',
│   │   'SPEED': '0',
│   │   'COURSE': '179',
│   │   'HEADING': '23',
│   │   'ELAPSED': '6',
│   │   'DESTINATION': 'NAKHODKA',
│   │   'FLAG': 'GW',
│   │   'LENGTH': '229',
│   │   'ROT': '0',
│   │   'SHIPNAME': 'WAN MING GONG ZHENG',
│   │   'SHIPTYPE': '7',
│   │   'SHIP_ID': '8856752',
│   │   'WIDTH': '36',
│   │   'L_FORE': '201',
│   │   'W_LEFT': '16',
│   │   'DWT': '86082',
│   │   'GT_SHIPTYPE': '6',
│   │   'TILE_X': '10',
│   │   'TILE_Y': '10',
│   │   'TILE_Z': '10'
│   },
│   '345265': {
│   │   'LAT': '42.68603',
│   │   'LON': '132.6497',
│   │   'SPEED': '51',
│   │   'COURSE': '279',
│   │   'HEADING'